In [90]:
%cd /kaggle/working/

/kaggle/working


In [91]:
import multiprocessing as mp
num_cpu = mp.cpu_count()
num_cpu

2

In [92]:
_exp_name = "CNN2"

In [93]:
# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import os
import json
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset
from tqdm import tqdm
import random
import itertools
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [94]:
myseed = 4012  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

In [95]:
test_tfm = transforms.Compose([
    # (height = width = 128)
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

train_tfm = transforms.Compose([
    # (height = width = 128)
    #transforms.CenterCrop()
    transforms.RandomResizedCrop((128, 128), scale=(0.7, 1.0)),
    #transforms.AutoAugment(transforms.AutoAugmentPolicy.IMAGENET),
    #transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.5),
    transforms.RandomRotation(180),
    transforms.RandomAffine(30),
    #transforms.RandomInvert(p=0.2),
    #transforms.RandomPosterize(bits=2),
    #transforms.RandomSolarize(threshold=192.0, p=0.2),
    #transforms.RandomEqualize(p=0.2),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    #transforms.RandomApply(torch.nn.ModuleList([]))
])

In [96]:
class FoodDataset(Dataset):

    def __init__(self,path,tfm=test_tfm,files = None):
        super(FoodDataset).__init__()
        # print(path)
        self.path = path
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
        if files != None:
            self.files = files
        print(f"One {path} sample",self.files[0])
        self.transform = tfm
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)
        #im = self.data[idx]
        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label
        return im,label

In [97]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input dimension [3, 128, 128]
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  # [64, 128, 128]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [64, 64, 64]

            nn.Conv2d(64, 128, 3, 1, 1), # [128, 64, 64]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 32, 32]

            nn.Conv2d(128, 256, 3, 1, 1), # [256, 32, 32]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 16, 16]

            nn.Conv2d(256, 512, 3, 1, 1), # [512, 16, 16]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 8, 8]
            
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 8, 8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 4, 4]
            
            nn.Flatten()
        )
        self.fc1 = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(512*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU()
        )
        self.fc2 = nn.Linear(512, 11)

    def forward(self, x):
        out = self.cnn(x)
        out = self.fc1(out)
        return self.fc2(out)

In [98]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [99]:
batch_size = 128
_dataset_dir = "/kaggle/input/food-11/food-11"

train_set = FoodDataset(os.path.join(_dataset_dir,"training"), tfm=train_tfm)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=num_cpu, pin_memory=True)
valid_set = FoodDataset(os.path.join(_dataset_dir,"validation"), tfm=test_tfm)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=num_cpu, pin_memory=True)
test_set = FoodDataset(os.path.join(_dataset_dir,"test"), tfm=test_tfm)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=num_cpu, pin_memory=True)

One /kaggle/input/food-11/food-11/training sample /kaggle/input/food-11/food-11/training/0_0.jpg
One /kaggle/input/food-11/food-11/validation sample /kaggle/input/food-11/food-11/validation/0_0.jpg
One /kaggle/input/food-11/food-11/test sample /kaggle/input/food-11/food-11/test/0_0.jpg


In [101]:
device = "cuda" if torch.cuda.is_available() else "cpu"
checkpoint = torch.load(f"/kaggle/input/cnn-ck2/{_exp_name}_best.ckpt")

model_best = Classifier().to(device)
model_best.load_state_dict(checkpoint)
model_best.eval()
model_best.cnn.register_forward_hook(get_activation('cnn'))
model_best.fc1.register_forward_hook(get_activation('fc1'))

X_train_noFC = []
X_valid_noFC = []
X_test_noFC = []

X_train_FC = []
X_valid_FC = []
X_test_FC = []

y_train_XGB = []
y_valid_XGB = []
y_test_XGB = []

train_accs = []
valid_accs = []
test_accs = []

with torch.no_grad():
    
    for batch in tqdm(train_loader):
        data, labels = batch 
        train_pred = model_best(data.to(device))
        train_label = np.argmax(train_pred.cpu().data.numpy(), axis=1)
        X_train_noFC.extend(activation['cnn'].tolist())
        X_train_FC.extend(activation['fc1'].tolist())
        y_train_XGB.extend(labels.tolist())
        acc = (train_pred.argmax(dim=-1) == labels.to(device)).float().mean()
        train_accs.append(acc)
        
    for batch in tqdm(valid_loader):
        data, labels = batch 
        valid_pred = model_best(data.to(device))
        valid_label = np.argmax(valid_pred.cpu().data.numpy(), axis=1)
        X_valid_noFC.extend(activation['cnn'].tolist())
        X_valid_FC.extend(activation['fc1'].tolist())
        y_valid_XGB.extend(labels.tolist())
        acc = (valid_pred.argmax(dim=-1) == labels.to(device)).float().mean()
        valid_accs.append(acc)
        
    for batch in tqdm(test_loader):
        data, labels = batch
        test_pred = model_best(data.to(device))
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        X_test_noFC.extend(activation['cnn'].tolist())
        X_test_FC.extend(activation['fc1'].tolist())
        y_test_XGB.extend(labels.tolist())
        acc = (test_pred.argmax(dim=-1) == labels.to(device)).float().mean()
        test_accs.append(acc)
        
train_acc = sum(train_accs) / len(train_accs)
valid_acc = sum(valid_accs) / len(valid_accs)
test_acc = sum(test_accs) / len(test_accs)

print(f"Train accurary: {train_acc}")
print(f"Valid accurary: {valid_acc}")
print(f"Test accurary: {test_acc}")

100%|██████████| 27/27 [00:29<00:00,  1.07s/it]

Train accurary: 0.7773036956787109
Valid accurary: 0.703005850315094
Test accurary: 0.7255573868751526


In [102]:
X_train_noFC = np.array(X_train_noFC)
X_train_FC = np.array(X_train_FC)
y_train_XGB = np.array(y_train_XGB)

X_valid_noFC = np.array(X_valid_noFC)
X_valid_FC = np.array(X_valid_FC)
y_valid_XGB = np.array(y_valid_XGB)

X_test_noFC = np.array(X_test_noFC)
X_test_FC = np.array(X_test_FC)
y_test_XGB = np.array(y_test_XGB)

In [103]:
xgb_train_noFC = xgb.DMatrix(data=X_train_noFC, label=y_train_XGB)
xgb_valid_noFC = xgb.DMatrix(data=X_valid_noFC, label=y_valid_XGB)
xgb_test_noFC = xgb.DMatrix(data=X_test_noFC, label=y_test_XGB)

xgb_train_FC = xgb.DMatrix(data=X_train_FC, label=y_train_XGB)
xgb_valid_FC = xgb.DMatrix(data=X_valid_FC, label=y_valid_XGB)
xgb_test_FC = xgb.DMatrix(data=X_test_FC, label=y_test_XGB)

In [104]:
params = {
    'eta': [0.001, 0.005, 0.01, 0.05],
    'gamma': [0, 0.5, 1.0],
    'subsample': [0.5, 0.8],
    'colsample_bytree': [0.5, 0.8],
    'colsample_bylevel': [0.5, 0.8],
    'max_depth': [5, 8, 10],
    'min_child_weight': [0, 2, 5]
}

In [105]:
params_list = list(itertools.product(*params.values()))

In [160]:
xgb.set_config(verbosity=0)
best_score_noFC = 10
best_param_noFC = ''

for params in params_list:
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        'num_class': 11,
        'tree_method': 'gpu_hist',
        'eta': params[0],
        'gamma': params[1],
        'subsample': params[2],
        'colsample_bytree': params[3],
        'colsample_bylevel': params[4],
        'max_depth': params[5],
        'min_child_weight': params[6]
    }
    
    print(f'{params} start')
    
    model_noFC = xgb.train(param, xgb_train_noFC, 
                           num_boost_round=400,
                           evals=[(xgb_train_noFC, 'train'), (xgb_valid_noFC, 'valid')],
                           early_stopping_rounds=10,
                           verbose_eval=False)
    
    if model_noFC.best_score < best_score_noFC:
        print('new best model')
        best_score_noFC = model_noFC.best_score
        best_param_noFC = json.loads(model_noFC.save_config())['learner']['gradient_booster']['updater']['grow_gpu_hist']['train_param']
    
    print(f'{params} end')

(0.001, 0, 0.5, 0.5, 0.5, 5, 0) start
new best model
(0.001, 0, 0.5, 0.5, 0.5, 5, 0) end


In [161]:
# valid mlogloss
best_score_noFC

2.3807368771327826

In [162]:
best_param_noFC

{'alpha': '0',
 'cache_opt': '1',
 'colsample_bylevel': '0.5',
 'colsample_bynode': '1',
 'colsample_bytree': '0.5',
 'eta': '0.00100000005',
 'gamma': '0',
 'grow_policy': 'depthwise',
 'interaction_constraints': '',
 'lambda': '1',
 'learning_rate': '0.00100000005',
 'max_bin': '256',
 'max_cat_to_onehot': '4',
 'max_delta_step': '0',
 'max_depth': '5',
 'max_leaves': '0',
 'min_child_weight': '0',
 'min_split_loss': '0',
 'monotone_constraints': '()',
 'refresh_leaf': '1',
 'reg_alpha': '0',
 'reg_lambda': '1',
 'sampling_method': 'uniform',
 'sketch_eps': '0.0299999993',
 'sketch_ratio': '2',
 'sparse_threshold': '0.20000000000000001',
 'subsample': '0.5'}

In [163]:
# test mlogloss
sum(np.argmax(model_noFC.predict(xgb_test_noFC), axis=1) == y_test_XGB)/len(y_test_XGB)

0.6459515984463698

In [164]:
xgb.set_config(verbosity=0)
best_score_FC = 10
best_param_FC = ''

for params in params_list:
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        'num_class': 11,
        'tree_method': 'gpu_hist',
        'eta': params[0],
        'gamma': params[1],
        'subsample': params[2],
        'colsample_bytree': params[3],
        'colsample_bylevel': params[4],
        'max_depth': params[5],
        'min_child_weight': params[6]
    }
    
    print(f'{params} start')
    
    model_FC = xgb.train(param, xgb_train_FC, 
                         num_boost_round=400,
                         evals=[(xgb_train_FC, 'train'), (xgb_valid_FC, 'valid')],
                         early_stopping_rounds=10,
                         verbose_eval=False)
    
    if model_FC.best_score < best_score_FC:
        print('new best model')
        best_score_FC = model_FC.best_score
        best_param_FC = json.loads(model_FC.save_config())['learner']['gradient_booster']['updater']['grow_gpu_hist']['train_param']
    
    print(f'{params} end')

(0.001, 0, 0.5, 0.5, 0.5, 5, 0) start
new best model
(0.001, 0, 0.5, 0.5, 0.5, 5, 0) end


In [165]:
# valid mlogloss
best_score_FC

2.373168691671277

In [166]:
best_param_FC

{'alpha': '0',
 'cache_opt': '1',
 'colsample_bylevel': '0.5',
 'colsample_bynode': '1',
 'colsample_bytree': '0.5',
 'eta': '0.00100000005',
 'gamma': '0',
 'grow_policy': 'depthwise',
 'interaction_constraints': '',
 'lambda': '1',
 'learning_rate': '0.00100000005',
 'max_bin': '256',
 'max_cat_to_onehot': '4',
 'max_delta_step': '0',
 'max_depth': '5',
 'max_leaves': '0',
 'min_child_weight': '0',
 'min_split_loss': '0',
 'monotone_constraints': '()',
 'refresh_leaf': '1',
 'reg_alpha': '0',
 'reg_lambda': '1',
 'sampling_method': 'uniform',
 'sketch_eps': '0.0299999993',
 'sketch_ratio': '2',
 'sparse_threshold': '0.20000000000000001',
 'subsample': '0.5'}

In [167]:
# test mlogloss
sum(np.argmax(model_FC.predict(xgb_test_FC), axis=1) == y_test_XGB)/len(y_test_XGB)

0.7191514789363609